<a href="https://colab.research.google.com/github/bradleywjenks/CIVE_70019_70057/blob/main/notebooks/pressure_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pressure control implementations in water distribution networks
### CIVE 70019/70057
Department of Civil and Environmental Engineering, Imperial College London\
Autumn 2023

### Preamble
This notebook walks through the following pressure control implementations in water distribution network operations:
- Fixed outlet
- Time-based
- Flow modulation
- Model-based optimization

Code implementations for flow modulation and model-based optimal control will be presented.

First, we must clone the GitHub repository and install dependencies (only run this once).

In [6]:
# Run this cell if in Google Colab
!git clone https://github.com/bradleywjenks/CIVE_70019_70057.git
!pip install wntr

fatal: destination path 'CIVE_70019_70057' already exists and is not an empty directory.


### Data preparation

Load data provided in the module repository.

In [5]:
# # Load dependencies
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
import os
import scipy.io as sio
import numpy as np

data_dir = '/content/CIVE_70019_70057/data/operational_data/'
data_name = 'data.mat'

# Load the .mat file
mat_data = sio.loadmat(os.path.join(data_dir, data_name))

# Extract the variables you want to save as .npy
data_to_save = {
    'p_in': mat_data['p_in'],
    'p_out': mat_data['p_out'],
    'p_critical': mat_data['p_critical'],
    'flow': mat_data['flow'],
    'time': mat_data['time'],
    'outlet_elev': mat_data['outlet_elev'],
    'critical_elev': mat_data['critical_elev']
}

py_data = 'data.npy'
# Save the data as a .npy file
np.save(os.path.join(data_dir, data_name))

FileNotFoundError: ignored